In [1]:
# FIFO
class Queue:
    
    def __init__(self):
        self.queue = []
        
    def __str__(self):
        result = f"Number of items in queue = {len(self.queue)}\n"
        for item in self.queue:
            result += f"{item}\n"
        return result 
    
    def enqueue(self, node):
        self.queue.append(node)  
        
    def dequeue(self):
        if not self.empty():
            return self.queue.pop(0)
        else:
            print("The queue is empty.")
            
    def empty(self):
        return len(self.queue) == 0
    
    def contains_state(self, state):
        return any(((node.state.current_amount_of_water_in_pitcher_one == state.current_amount_of_water_in_pitcher_one) and (node.state.current_amount_of_water_in_pitcher_two == state.current_amount_of_water_in_pitcher_two)) for node in self.queue)

In [2]:
# q = Queue()

In [3]:
# q.enqueue("Anjana")

In [4]:
# q.enqueue("Aadi")

In [5]:
# q.dequeue()

In [6]:
# print(q)

In [7]:
# q.empty()

In [8]:
# q.dequeue()

In [9]:
# q.empty()

In [10]:
class Node:

    def __init__(self, state, parent, action, depth):
        self.state = state
        self.parent = parent
        self.action = action
        self.depth = depth
        
    def __str__(self):
        result = f"State: {self.state}"
        result += f" | Depth: {self.depth}"
        if self.parent != None:
            result += f" | Parent: {self.parent.state}"
            result += f" | Action: {self.action}"
        return result
    
    def repeated_state(self):
        if self.parent == None: 
            return 0
        if self.parent.parent == None: 
            return 0
        if self.parent.parent.state.equals(self.state): 
            return 1
        return 0

In [11]:
class Search:

    def __init__(self, initialState, goalState):
        self.q = Queue()
        self.q.enqueue(Node(initialState, None, None, 0))
        self.goalState = goalState
            
    def solve(self):
        
        # Initialize an empty explored set
        self.explored = set()
        
        while not self.q.empty():
            current = self.q.dequeue()
            if self.goalState.equals(current.state):
                return current
            
            # Mark node as explored
            self.explored.add(current.state)
            
            successors = current.state.apply_actions()
            actions = current.state.action_names()
            for i in range(len(successors)):
                if not self.q.contains_state(successors[i]) and successors[i] not in self.explored and not successors[i].illegal():
                    n = Node(successors[i],
                            current,
                            actions[i],
                            current.depth+1)
                    if n.repeated_state():
                        del(n)
                    else:
                        self.q.enqueue(n)
                        #print(self.q)
        return None
    
    def show_solution_helper(self, node):
        result = []
        while node != None:
            result.insert(0, node)
            node = node.parent
        return result
    
    def show_solution(self, node):
        path = self.show_solution_helper(node)        
        for current in path:
            if current.depth != 0:
                print(f"Step {current.depth}:", current.action)
            print(current.state)
        print(f"Solution requires {current.depth} steps")
    
    def find_solution(self):
        solution = self.solve()
        if solution == None:
            print("No solution!!")
        else:
            self.show_solution(solution)

In [12]:
class PitcherState:
    
    global pitcher_one_capacity
    global pitcher_two_capacity

    def __init__(self, current_amount_of_water_in_pitcher_one, current_amount_of_water_in_pitcher_two):
        self.current_amount_of_water_in_pitcher_one = current_amount_of_water_in_pitcher_one
        self.current_amount_of_water_in_pitcher_two = current_amount_of_water_in_pitcher_two
        
    def __str__(self):
        return "("+str(self.current_amount_of_water_in_pitcher_one)+","+str(self.current_amount_of_water_in_pitcher_two)+")"
    
    def illegal(self):
        if self.current_amount_of_water_in_pitcher_one < 0 or self.current_amount_of_water_in_pitcher_two < 0: return 1
        if self.current_amount_of_water_in_pitcher_one > pitcher_one_capacity or self.current_amount_of_water_in_pitcher_two > pitcher_two_capacity: return 1
        return 0
    
    def equals(self, state):
        return self.current_amount_of_water_in_pitcher_one==state.current_amount_of_water_in_pitcher_one and self.current_amount_of_water_in_pitcher_two==state.current_amount_of_water_in_pitcher_two
    
    def fill_pitcher_one(self):
        return PitcherState(pitcher_one_capacity, self.current_amount_of_water_in_pitcher_two)
    
    def fill_pitcher_two(self):
        return PitcherState(self.current_amount_of_water_in_pitcher_one, pitcher_two_capacity)
    
    def drain_pitcher_one(self):
        return PitcherState(0, self.current_amount_of_water_in_pitcher_two)
    
    def drain_pitcher_two(self):
        return PitcherState(self.current_amount_of_water_in_pitcher_one, 0)
    
    def pour_pitcher_one_to_pitcher_two(self):
        capacity = pitcher_two_capacity - self.current_amount_of_water_in_pitcher_two
        if self.current_amount_of_water_in_pitcher_one > capacity:
            return PitcherState(self.current_amount_of_water_in_pitcher_one-capacity, pitcher_two_capacity)
        else:
            return PitcherState(0, self.current_amount_of_water_in_pitcher_two + self.current_amount_of_water_in_pitcher_one)
        
    def pour_pitcher_two_to_pitcher_one(self):
        capacity = pitcher_one_capacity - self.current_amount_of_water_in_pitcher_one
        if self.current_amount_of_water_in_pitcher_two > capacity:
            return PitcherState(pitcher_one_capacity, self.current_amount_of_water_in_pitcher_two-capacity)
        else:
            return PitcherState(self.current_amount_of_water_in_pitcher_one + self.current_amount_of_water_in_pitcher_two, 0)
        
    def action_names(self):
        return ["fill_pitcher_one", "fill_pitcher_two", "drain_pitcher_one", "drain_pitcher_two",
                "pour_pitcher_one_to_pitcher_two", "pour_pitcher_two_to_pitcher_one"]
    
    def apply_actions(self):
        return [self.fill_pitcher_one(), self.fill_pitcher_two(),
                self.drain_pitcher_one(), self.drain_pitcher_two(),
                self.pour_pitcher_one_to_pitcher_two(), self.pour_pitcher_two_to_pitcher_one()] 

In [13]:
pitcher_one_capacity = 3
pitcher_two_capacity = 3
pitcher_one_final_amount_of_water = 0
pitcher_two_final_amount_of_water = 2

if (pitcher_one_final_amount_of_water > pitcher_one_capacity) or (pitcher_two_final_amount_of_water > pitcher_two_capacity):
    print("No solution!!")
else:
    s = Search(PitcherState(0, 0), 
               PitcherState(pitcher_one_final_amount_of_water, pitcher_two_final_amount_of_water))  
    s.find_solution()

KeyboardInterrupt: 